# PRDualRank( ) and its Helpers

## QuestR Code

In [12]:
def computePatternRecallFromTuple(sg, total_eids, eids2Sup, eid2recall, eidsFeatureCount):
      tmp_sum = 0.0
      for eid in range(total_eids):
        if eids2Sup[eid] > 0:
          tmp_sum += eid2recall.get(eid, 0) * eidsFeatureCount[eid].get(sg, 0) / eids2Sup[eid]
      return tmp_sum

def computeTupleRecallFromPattern(eid, extr_patterns, eidsFeatureCount, patterns2Sup, pattern2recall):
      tmp_sum = 0.0
      for sg in extr_patterns:
        tmp_sum += pattern2recall.get(sg, 0) * eidsFeatureCount[eid].get(sg, 0) / patterns2Sup[sg]
      return tmp_sum

def doInferenceRecallExtPatternOnly(extr_patterns, pattern2recall, total_eids, eid2recall, 
                                    eids2Sup, eidsFeatureCount, patterns2Sup):
      for sg in extr_patterns:
        pattern2recall[sg] = computePatternRecallFromTuple(sg, total_eids, eids2Sup, 
                                                           eid2recall, eidsFeatureCount)
      for eid in range(total_eids):
        eid2recall[eid] = computeTupleRecallFromPattern(eid, extr_patterns, eidsFeatureCount, 
                                                        patterns2Sup, pattern2recall)

## QuestP Code

In [11]:
def computePatternPrecisionFromTuple(p, pattern2eids, eidsFeatureCount, 
                                     pattern2precision, eid2precision, patterns2Sup):
      tmp_sum = 0.0
      for eid in pattern2eids[p]:
        sup = eidsFeatureCount[eid][p]
        tmp_sum += eid2precision[eid]*sup
      total = patterns2Sup[p]
      if total > 0:
        return tmp_sum / total
      else: return 0.0

def computeTuplePrecisionFromPattern(eid, groundTruthEids, eid2precision, 
                                     extr_patterns, eidsFeatureCount, eid2patterns, 
                                     pattern2precision, eids2Sup):
      if eid in groundTruthEids:
        return eid2precision[eid]
      tmp_sum = 0.0
      for sg in eid2patterns[eid]:
        sup = eidsFeatureCount[eid].get(sg, 0)
        tmp_sum += pattern2precision[sg]*sup
      total = eids2Sup[eid]
      if total > 0:
        return tmp_sum / total
      else: return 0.0

def doInferencePrecExtPatternOnly(extr_patterns, pattern2eids, eidsFeatureCount, 
                                  pattern2precision, eid2precision, patterns2Sup, 
                                  total_eids, groundTruthEids, eid2patterns, eids2Sup):
      for sg in extr_patterns:
        pattern2precision[sg] = computePatternPrecisionFromTuple(sg, pattern2eids, eidsFeatureCount, 
                                                                 pattern2precision, eid2precision, patterns2Sup)

      for eid in range(total_eids):
        if eid not in groundTruthEids:
          eid2precision[eid] = computeTuplePrecisionFromPattern(eid, groundTruthEids, eid2precision,extr_patterns, 
                                                                eidsFeatureCount, eid2patterns, pattern2precision, 
                                                                eids2Sup)

## The PRDualRank( ) Function

In [9]:
# expand the set of seedEntities and return eids by order, excluding seedEntities (original children)
def prDualRank(seedEidsWithConfidence, negativeSeedEids, eid2patterns, pattern2eids, eidAndPattern2strength,
             eid2types, type2eids, eidAndType2strength, eid2ename, eidsFeatureCount, eids2Sup, patterns2Sup, 
             FLAGS_VERBOSE=False, FLAGS_DEBUG=False):
      ''' 

      :param seedEidsWithConfidence: a list of [eid (int), confidence_score (float)]
      :param negativeSeedEids: a set of eids (int) that should not be included
      :param eid2patterns:
      :param pattern2eids:
      :param eidAndPattern2strength:
      :param eid2types:
      :param type2eids:
      :param eidAndType2strength:
      :param eid2ename:

      :return: P and T sorted by precision and recall
      '''
      print(patterns2Sup)
      seedEids = [ele[0] for ele in seedEidsWithConfidence]
      groundTruthEids = [ele[0] for ele in seedEidsWithConfidence]
      eid2confidence = {ele[0]: ele[1] for ele in seedEidsWithConfidence}

      ## Cache the seedEids for later use
      cached_seedEids = set([ele for ele in seedEids])
      if FLAGS_VERBOSE:
        print('Seed set:')
        for eid in seedEids:
          print(eid, eid2ename[eid])
        print("[INFO] Start SetExpan")

    ## establish ground truth
    ##double but not int
      total_eids = len(eid2ename)
      eid2precision = {x: 0.0 for x in range(total_eids)}
      for eid in seedEids:
        eid2precision[eid] = 1.0
      tmp_sum = len(seedEids)
      eid2recall = {eid: eid2precision[eid]/tmp_sum for eid in seedEids}

      extr_patterns = list(pattern2eids.keys())

      pattern2precision = {sg: 0.0 for sg in extr_patterns}
      pattern2recall = {sg: 0.0 for sg in extr_patterns}


      iters = 0
      while iters < MAX_ITER_SET:
        iters += 1
        prev_seeds = set(seedEids)
        start = time.time()

        #QuestP till convergence
        doInferencePrecExtPatternOnly(extr_patterns, pattern2eids, eidsFeatureCount, 
                                      pattern2precision, eid2precision, patterns2Sup, 
                                      total_eids, groundTruthEids, eid2patterns, eids2Sup)
        #QuestR till convergence
        doInferenceRecallExtPatternOnly(extr_patterns, pattern2recall, total_eids, 
                                        eid2recall, eids2Sup, eidsFeatureCount, patterns2Sup)


        all_end = time.time()

        if FLAGS_DEBUG:
          print("End of iteration %s" % iters)

      expanded_eid_pre = sorted(eid2precision, key=eid2precision.__getitem__, reverse=True)
      expanded_eid_rec = sorted(eid2recall, key=eid2recall.__getitem__, reverse=True)
      expanded_pattern_pre = sorted(pattern2precision, key=pattern2precision.__getitem__, reverse=True)

      expanded_pattern_rec = sorted(pattern2recall, key=pattern2recall.__getitem__, reverse=True)

      return expanded_pattern_pre, expanded_pattern_rec, expanded_eid_pre, expanded_eid_rec
